In [ ]:
#API_BASE = 'https://www.bitmex.com/api/v1/'



connector = bitmex.BitMEX(base_url=API_BASE,apiKey=API_KEY,apiSecret=API_SECRET)

#connector = bitmex.BitMEX(base_url=API_BASE)

def GetOrderBookPrice():
    queryL2 = {
    'symbol': 'XBTUSD',
    'depth' : 1,
    }

    dataL2 = pd.DataFrame(connector._curl_bitmex(path="orderbook/L2", verb="GET", query=queryL2, timeout=5))
    BuySide = dataL2.price[1]
    SellSide = dataL2.price[0]
    return BuySide

def GetOrderBookPriceEnd():
    queryL2 = {
    'symbol': 'XBTUSD',
    'depth' : 1,
    }

    dataL2 = pd.DataFrame(connector._curl_bitmex(path="orderbook/L2", verb="GET", query=queryL2, timeout=5))
    BuySide = dataL2.price[1]
    SellSide = dataL2.price[0]
    return SellSide

def Leverage(leverage):
    query = {
    'symbol': 'XBTUSD',
    'leverage' : leverage,
    }
    data = connector._curl_bitmex(path="position/leverage", verb="POST", query=query, timeout=5)

    
def BeginShort(GetOrderBookPrice, Balance):
    queryShort = {
        'symbol': 'XBTUSD',
        'orderQty': Balance,
        'price' : GetOrderBookPrice(),
    }
    data = connector._curl_bitmex(path="order", verb="POST", query=queryShort, timeout=5)
    
def EndShortMkt(Balance):
    queryShort = {
        'symbol': 'XBTUSD',
        'orderQty': -1*Balance,
        'ordType': 'Market',
        'execInst': 'Close'
    }
    data = connector._curl_bitmex(path="order", verb="POST", query=queryShort, timeout=5)
    
def EndShortLimit(GetOrderBookPriceEnd, Balance):
    queryShort = {
        'symbol': 'XBTUSD',
        'orderQty': -1*Balance,
        'price': GetOrderBookPriceEnd(),
        'execInst': 'Close'
    }
    data = connector._curl_bitmex(path="order", verb="POST", query=queryShort, timeout=5)


## RSI w/ ALPHA ##
def rsi_ewma(trades, name):
    # Create connector
    connector = bitmex.BitMEX(base_url=API_BASE)
    # Do trade history query
    path = "trade/bucketed?binSize=1m&symbol=XBTUSD&columns=symbol%2C%20timestamp%2C%20close"
    count = 500  # max API will allow
    query = {
        'reverse': 'true',
        'start': 0,
        'count': count,
        }
    csvwriter = None
    with open(name, 'w+') as f:
        while True:
            data = connector._curl_bitmex(path=path, verb="GET", query=query, timeout=5)
            if csvwriter is None:
                # csv requires dict keys
                if len(data) == 0:
                    break
                keys = sorted(data[0].keys())
                # Write to stdout
                csvwriter = csv.DictWriter(f, fieldnames=keys)
                csvwriter.writeheader()

            csvwriter.writerows(data)
            #data2 = pd.DataFrame
            data2 = np.append(data, data)
            query['start'] += count
            if query['start'] > end:
                break
            if len(data) < count:
                break

    df1 = pd.read_csv(name)
    df_all = pd.DataFrame(df1)
    df_all = pd.DataFrame(df_all.values[::-1], df_all.index, df_all.columns) # turn around data

    close = df_all['close'] #close values
    vwap = pd.Series(df_all['vwap']) #volume-weighted price average
    l = len(close)
    rsi_len = 15
    ## RSI CLOSE ##
    change_close = pd.Series(np.zeros(l))
    gain = pd.Series(np.zeros(l))
    loss = pd.Series(np.zeros(l))
    smagain = pd.Series(np.zeros(l))
    smaloss = pd.Series(np.zeros(l))
    RS = pd.Series(np.zeros(l))
    RSIclose = pd.Series(np.zeros(l))
    for i in range(1,len(close)):   
        #print(close[i])
        change_close[i] = close[i]-close[i-1]
        if change_close[i] > 0:
            gain[i] = change_close[i]
        elif change_close[i] < 0:
            loss[i] = change_close[i]*-1
        elif change_close[i] == 0.00000:
            loss[i] = 0
            gain[i] = 0
        if i == rsi_len:   
            smagain[rsi_len] = gain[i-rsi_len:i].sum()/rsi_len
            smaloss[rsi_len] = loss[i-rsi_len:i].sum()/rsi_len
        elif i > rsi_len: 
            smagain[i] = (1-alpha)*smagain[i-1]+(alpha)*gain[i]
            smaloss[i] = (1-alpha)*smaloss[i-1]+(alpha)*loss[i]
            if smaloss[i] == 0:
                RSIclose[i] = 100
            else:
                RS[i] = smagain[i]/smaloss[i]
                RSIclose[i] = 100 - 100/(1+RS[i])
               #print(RSIclose[i])
    return RSIclose